In [1]:
# %pip install git+https://github.com/tensorflow/docs

import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import RiotWatcher, LolWatcher, ApiError
from pymongo import MongoClient

from utils import configuration
from utils.utils import *

settings = configuration.settings
api_key = settings.api_key

In [2]:
lol_watcher = LolWatcher(api_key)
watcher = RiotWatcher(api_key)

my_region = 'na1'
my_summoner = 'insanity123'

me = lol_watcher.summoner.by_name(my_region, my_summoner)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

[]


In [3]:
me

{'id': 'jg7S93hSw4KfuYyaks9xYzAFZ4w3bp_PSfT4LsHJi60A-SQ',
 'accountId': 'Ab7L0SV8v_nX7hVWwKV7IbEAbA0dj-C4e9PvMcPaU-7hf8o',
 'puuid': 'K8izf-1-ovMV0xlJ2qhoL7Amw98UG248C2xf8GGDVSqf73KklNrVoNN1PQe1R4Zb2JFsOFa_kSEzpg',
 'name': 'fury hawkx',
 'profileIconId': 745,
 'revisionDate': 1656557016000,
 'summonerLevel': 20}

In [4]:
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']
summoner_spells_version=versions['n']['summoner']
items_version=versions['n']['item']

current_champ_list = lol_watcher.data_dragon.champions(champions_version)

my_matches = lol_watcher.match.matchlist_by_puuid(my_region, me['puuid'], count=100)

In [5]:
len(my_matches)

9

In [6]:
last_match01 = lol_watcher.match.by_id(my_region,      my_matches[0])

In [7]:
# First we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']

# Lets get some champions
static_champ_list = lol_watcher.data_dragon.champions(champions_version)
static_summoners_list = lol_watcher.data_dragon.summoner_spells(champions_version)
# print(current_champ_list)

In [8]:
last_match01 #['info']['participants']

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_3747820277',
  'participants': ['ELOe8U6_n-Js9RfHLtDs4rDJuP_O4f3XrckXPnD0nnf0rdf1FMrAzgCAXAKSk70y4wyjzhJ4mCUu3A',
   'juouUq8MyAgVYaTnxtTCG2Xn8ir9vmXfgnKUgfnT-LRLZ_-NYlAlrChHSSl-ix59Mq-J9qrHDdZHuA',
   'tdZA5-QO4JHdRM_A8ooCbWABZf9sOwp_L1wliAODGb9GcPamlT-_KX9WMe-nyUROelxMTdBMEF9sIg',
   'K8izf-1-ovMV0xlJ2qhoL7Amw98UG248C2xf8GGDVSqf73KklNrVoNN1PQe1R4Zb2JFsOFa_kSEzpg',
   'whAWIYse07YFwvgeDIcSTnHpOl50UbaWRPdzLYw4KkDXKB299tOWDwrQUz5RPmhgomQHrM1fWRvasQ']},
 'info': {'gameCreation': 1610896033000,
  'gameDuration': 1289211,
  'gameId': 3747820277,
  'gameMode': 'CLASSIC',
  'gameName': 'teambuilder-match-3747820277',
  'gameStartTimestamp': 1610896146842,
  'gameType': 'MATCHED_GAME',
  'gameVersion': '11.1.352.5559',
  'mapId': 11,
  'participants': [{'assists': 10,
    'baronKills': 0,
    'bountyLevel': 7,
    'champExperience': 8797,
    'champLevel': 12,
    'championId': 37225015,
    'championName': 'DrMundo',
    'championTransform'

In [34]:
# static_summoners_list

In [35]:
champ_dict = {}
for key in static_champ_list['data']:
    row = static_champ_list['data'][key]
    champ_dict[row['key']] = row['id']

In [57]:
champ_dict

{'266': 'Aatrox',
 '103': 'Ahri',
 '84': 'Akali',
 '166': 'Akshan',
 '12': 'Alistar',
 '32': 'Amumu',
 '34': 'Anivia',
 '1': 'Annie',
 '523': 'Aphelios',
 '22': 'Ashe',
 '136': 'AurelionSol',
 '268': 'Azir',
 '432': 'Bard',
 '200': 'Belveth',
 '53': 'Blitzcrank',
 '63': 'Brand',
 '201': 'Braum',
 '51': 'Caitlyn',
 '164': 'Camille',
 '69': 'Cassiopeia',
 '31': 'Chogath',
 '42': 'Corki',
 '122': 'Darius',
 '131': 'Diana',
 '119': 'Draven',
 '36': 'DrMundo',
 '245': 'Ekko',
 '60': 'Elise',
 '28': 'Evelynn',
 '81': 'Ezreal',
 '9': 'Fiddlesticks',
 '114': 'Fiora',
 '105': 'Fizz',
 '3': 'Galio',
 '41': 'Gangplank',
 '86': 'Garen',
 '150': 'Gnar',
 '79': 'Gragas',
 '104': 'Graves',
 '887': 'Gwen',
 '120': 'Hecarim',
 '74': 'Heimerdinger',
 '420': 'Illaoi',
 '39': 'Irelia',
 '427': 'Ivern',
 '40': 'Janna',
 '59': 'JarvanIV',
 '24': 'Jax',
 '126': 'Jayce',
 '202': 'Jhin',
 '222': 'Jinx',
 '145': 'Kaisa',
 '429': 'Kalista',
 '43': 'Karma',
 '30': 'Karthus',
 '38': 'Kassadin',
 '55': 'Katarina',


In [36]:
summoners_dict = {}
for key in static_summoners_list['data']:
    row = static_summoners_list['data'][key]
    summoners_dict[row['key']] = row['id']

In [37]:
summoners_dict

{'21': 'SummonerBarrier',
 '1': 'SummonerBoost',
 '14': 'SummonerDot',
 '3': 'SummonerExhaust',
 '4': 'SummonerFlash',
 '6': 'SummonerHaste',
 '7': 'SummonerHeal',
 '13': 'SummonerMana',
 '30': 'SummonerPoroRecall',
 '31': 'SummonerPoroThrow',
 '11': 'SummonerSmite',
 '39': 'SummonerSnowURFSnowball_Mark',
 '32': 'SummonerSnowball',
 '12': 'SummonerTeleport',
 '54': 'Summoner_UltBookPlaceholder',
 '55': 'Summoner_UltBookSmitePlaceholder'}

In [69]:
n_games = 10
Games = {}
Game_duration = np.zeros(n_games)
Damage = np.zeros(n_games)
Gold = []
match_detail = None


j = 0
cont = 0
while cont < n_games:
    try:
        last_match = my_matches[cont]
        match_detail = lol_watcher.match.by_id(my_region, last_match)

        participants = []
        for row in match_detail['info']['participants']:
            participants_row = {}
            participants_row['champion'] = row['championId']
            participants_row['win'] = row['kills']
            participants_row['assists'] = row['assists']
            participants.append(participants_row)
            
        Games[j] = pd.DataFrame(participants)

        Summoner_name = []
        for row in match_detail['info']['participants']:
            Summoner_name_row = {}
            Summoner_name_row = row['summonerName']
            Summoner_name.append(Summoner_name_row)

        i = 0
        for row in participants:
            row['championName'] = champ_dict[str(row['champion'])]
            row['Summoner_name'] = Summoner_name[i]
            row['Summoner Spell 1'] = summoners_dict[str(row['summoner1Id'])]
            row['Summoner Spell 2'] = summoners_dict[str(row['summoner2Id'])]
            i += 1

        Games[j] = pd.DataFrame(participants)
        for index, row in Games[j].iterrows():
            if row['Summoner_name'] == my_summoner:
                Damage[j] = row['totalDamageDealt']
                Gold[j] = row['goldEarned']

        time.sleep(1)
        j += 1
        cont += 1

    except:
        cont += 1


In [80]:
Games[0]

,champion,win,assists
0,84,2,7
1,76,3,8
2,236,8,3
3,81,6,4
4,555,8,5
5,79,12,8
6,421,15,8
7,142,6,10
8,202,2,14
9,147,0,16


In [39]:
# last_match = my_matches[0]
# match_detail = lol_watcher.match.by_id(my_region, last_match)

In [40]:
Games

{0:    champion  win  assists
 0        84    2        7
 1        76    3        8
 2       236    8        3
 3        81    6        4
 4       555    8        5
 5        79   12        8
 6       421   15        8
 7       142    6       10
 8       202    2       14
 9       147    0       16}

In [2]:
client = MongoClient(settings.db_uri)
db = client.admin


In [3]:
serverStatusResult=db.command("serverStatus")

In [4]:
serverStatusResult

{'host': 'd335cf0b320e',
 'version': '5.0.9',
 'process': 'mongod',
 'pid': 1,
 'uptime': 192.0,
 'uptimeMillis': 192268,
 'uptimeEstimate': 192,
 'localTime': datetime.datetime(2022, 7, 28, 3, 18, 2, 355000),
 'asserts': {'regular': 0,
  'warning': 0,
  'msg': 0,
  'user': 0,
  'tripwire': 0,
  'rollovers': 0},
 'catalogStats': {'collections': 0,
  'capped': 0,
  'timeseries': 0,
  'views': 0,
  'internalCollections': 4,
  'internalViews': 0},
 'connections': {'current': 4,
  'available': 838856,
  'totalCreated': 4,
  'active': 2,
  'threaded': 4,
  'exhaustIsMaster': 0,
  'exhaustHello': 0,
  'awaitingTopologyChanges': 1},
 'electionMetrics': {'stepUpCmd': {'called': 0, 'successful': 0},
  'priorityTakeover': {'called': 0, 'successful': 0},
  'catchUpTakeover': {'called': 0, 'successful': 0},
  'electionTimeout': {'called': 0, 'successful': 0},
  'freezeTimeout': {'called': 0, 'successful': 0},
  'numStepDownsCausedByHigherTerm': 0,
  'numCatchUps': 0,
  'numCatchUpsSucceeded': 0,
 

In [8]:
db = client.business

In [11]:
fivestar = db.reviews.find_one({'rating': 5})
print(fivestar)

{'_id': ObjectId('62dfa58d7cf460763f07c980'), 'name': 'Tastey Sandwich Corporation', 'rating': 5, 'cuisine': 'American'}


In [16]:
fivestarcount = db.reviews.count_documents({'rating': 5})
print(fivestarcount)

110


In [17]:
print('\nThe sum of each rating occurance across all data grouped by rating ')
stargroup=db.reviews.aggregate(
# The Aggregation Pipeline is defined as an array of different operations
[
# The first stage in this pipe is to group data
{ '$group':
    { '_id': "$rating",
     "count" : 
                 { '$sum' :1 }
    }
},
# The second stage in this pipe is to sort the data
{"$sort":  { "_id":1}
}
# Close the array with the ] tag             
] )
# Print the result
for group in stargroup:
    print(group)


The sum of each rating occurance across all data grouped by rating 
{'_id': 1, 'count': 102}
{'_id': 2, 'count': 91}
{'_id': 3, 'count': 102}
{'_id': 4, 'count': 95}
{'_id': 5, 'count': 110}


In [18]:
ASingleReview = db.reviews.find_one({})
print('A sample document:')
print(ASingleReview)

result = db.reviews.update_one({'_id' : ASingleReview.get('_id') }, {'$inc': {'likes': 1}})
print('Number of documents modified : ' + str(result.modified_count))

UpdatedDocument = db.reviews.find_one({'_id':ASingleReview.get('_id')})
print('The updated document:')
print(UpdatedDocument)

A sample document:
{'_id': ObjectId('62dfa58d7cf460763f07c97f'), 'name': 'Big State LLC', 'rating': 1, 'cuisine': 'Italian'}
Number of documents modified : 1
The updated document:
{'_id': ObjectId('62dfa58d7cf460763f07c97f'), 'name': 'Big State LLC', 'rating': 1, 'cuisine': 'Italian', 'likes': 1}


In [20]:
result = db.restaurants.delete_many({"category": "Bar Food"})

In [21]:
result

In [3]:
client.close()

NameError: name 'client' is not defined

In [7]:
client = MongoClient(settings.db_uri)
db = client.tftchamp

In [8]:
SERVER: str = "kr"
LEAGUE: str = "challengers"

In [9]:
summoners_collection = db[f'{SERVER}_{LEAGUE}_summoners']
summoners_df = pd.DataFrame(list(summoners_collection.find()))

In [10]:
summoners_df

,_id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,8ygNfnWPnYvCFKDJnKik0RNHxbOSveC8zrpq9jmYAIcLJw,oFecdwjzXmg2mfLvluM9G01HF9WcLuP1yTOSp0Z-pjVl,5On3dmeoAoFhTS1wnsB4BoSm5FbUbbzSOL_JWjgoHo8Lvj...,ShrimpVicious,4804,1658850187000,312,8ygNfnWPnYvCFKDJnKik0RNHxbOSveC8zrpq9jmYAIcLJw,ShrimpVicious,773,I,203,169,False,False,False,False
1,xDrh71lkeyZz1M433ne4OAoFgeLuRJhPGrByX6uZLL3jgQ,jLpfV6VipK19F69tka6DoWtqia56_dRuvaMnOlw4fQi4,QPbpIbefMxNWNOQDlWt5f2OkznBFmsd-lzu8wZgYFb02oM...,띰 5,4558,1658892494000,414,xDrh71lkeyZz1M433ne4OAoFgeLuRJhPGrByX6uZLL3jgQ,띰 5,748,I,225,181,True,False,False,False
2,1oxOa7VMDXRGbXvefCT6mwQBH_LScFsQXuMfHhIgdtm5kms,kv2gZIWXvDAH1h6JS1-p7L5ExnxSt36kDTu1v5M2GfHabYk,i1dT8GjNU0serv2XnwiPFFeiN0C-fKO_pUSPvWuGytfG40...,서아 사랑해,4903,1658893752000,31,1oxOa7VMDXRGbXvefCT6mwQBH_LScFsQXuMfHhIgdtm5kms,서아 사랑해,774,I,194,158,False,False,True,False
3,M0iKCUaEBeo2hH0DGxio8B24BH2tEjKP6sdxFtujVkThDrI,gngNSNDCLLjwz7k_a6o3SX71xoDC-qoQ0I9cj-VqfKMA1W...,Bv2PeFBVV3PNNBEv_zyoHriFq3KqP3F0e8dFt5hroKI3Wb...,단간론파키리기리,3018,1658854021000,5,M0iKCUaEBeo2hH0DGxio8B24BH2tEjKP6sdxFtujVkThDrI,단간론파키리기리,1062,I,257,198,True,False,False,False
4,rXcfOv_yboB9INXZeQTPOiW7qe4ivN1U2AsEPPderBnXmNA,73MEi12a96QiN_zskIbIurrUb8mJRy2fYN6YM1KPsGaMzg...,F5VdDuzEn_kTiC9uvOGqjHg2cTiQn6i-GefH80AfrfaoRU...,롤좀하는가,5430,1658669362000,151,rXcfOv_yboB9INXZeQTPOiW7qe4ivN1U2AsEPPderBnXmNA,롤좀하는가,881,I,177,126,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,KNB2xsHVQewW2R610_hrWK66XbHJSejJ04qVcZs2twI3Ak...,PAJr53gS05LKAQF8OmIht-6yFbpIMpdNo4Jm4SApV4tdZN...,n_kDs6Qe_PI9pj8Y4vQqZ3FNGt_bTsNERJTxlCe9AL9xKH...,뇽녕냥뇽,29,1658894952000,1,KNB2xsHVQewW2R610_hrWK66XbHJSejJ04qVcZs2twI3Ak...,뇽녕냥뇽,1147,I,224,165,True,False,False,False
296,3UoA9CX2ceLbcoA6BxiWYPpC-d3Wer4V-T04DsYWAmVS4Ec,DHajpdSPOX6HZ9UcWprqaYnXp5naVh6U1vEYrC44k9qCb2w,mIdsBgM_RuKPhvg-C3rJLSWGuPQtCFRexANJJDnS8eOISm...,쿠퍼 머피,4571,1658766004000,18,3UoA9CX2ceLbcoA6BxiWYPpC-d3Wer4V-T04DsYWAmVS4Ec,쿠퍼 머피,726,I,100,76,False,False,True,False
297,v-g7ZIcypyxzsK9veF78p_V3NSav1Zr6vkPybwlBEEBu3eE,hpZrWFP3Qm4f8lBEtpI23gJ7dKH0o_ttk1Ll_WonK9Jq,zGL8kioRpsCIAkYcYBjxPZRLhOQIvbKN0DHb5WewZee-7j...,동물넝장,4896,1658813779000,162,v-g7ZIcypyxzsK9veF78p_V3NSav1Zr6vkPybwlBEEBu3eE,동물넝장,886,I,133,89,False,False,False,True
298,SW_RQ-1Xj7ojKdPJ-OvATJidRj_sac6pR6XGHnxmdX_wXwDj,9LYfXLvj_CJNlI2HyYW-WeoE7XSoHmje_9QF2HCGI2QGna...,4VVaPAvAMhI30iRmMrXKIiu12wFM_5SpuD71cto-tJZMkv...,난 감 한,5371,1658880998525,3,SW_RQ-1Xj7ojKdPJ-OvATJidRj_sac6pR6XGHnxmdX_wXwDj,난 감 한,677,I,173,130,False,False,False,False


In [26]:
puuid = summoners_df[:1]['puuid'].values

In [28]:
puuid[0]

'5On3dmeoAoFhTS1wnsB4BoSm5FbUbbzSOL_JWjgoHo8LvjcFmjiHC9YPQX9y3eUlvVfSYPUW_IQkdg'

In [11]:
matches_detail_collection = db[SERVER + '_' + 'matches_detail']
matches_asset: list = load_league_matches_db(matches_detail_collection, summoners_df)

123
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\furyx\AppData\Local\Temp\ipykernel_35492\225935193.py", line 2, in <cell line: 2>
    matches_asset: list = load_league_matches_db(matches_detail_collection, summoners_df)
  File "w:\workspace\tftchamp\backend\app\utils\utils.py", line 135, in load_league_matches_db
  File "w:\workspace\tftchamp\backend\app\utils\utils.py", line 123, in find_matches_db
    match_asset = find_collection_db(
  File "w:\workspace\tftchamp\backend\app\utils\utils.py", line 115, in find_collection_db
    return list(collection.find({key: value}))
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\pymongo\cursor.py", line 1248, in next
    if len(self.__data) or self._refresh():
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\pymongo\cursor.py", line 1165, in _refr

In [101]:
for _, summoner in summoners_df.iterrows():
    print(type(summoner['puuid']))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [96]:
SERVER

'kr'

In [ ]:
matches_asset

[]

In [99]:
list(matches_detail_collection.find({'info.participants.puuid': 'btPNzqiG4BTNjt6X0vP79pOzp5p6kN5_A4nIorA8Oub4zXN229kcOaMYYxekO2jQ1Fph0Y-zsima4w'}))

[{'_id': 'KR_6041556013',
  'metadata': {'data_version': '5',
   'match_id': 'KR_6041556013',
   'participants': ['btPNzqiG4BTNjt6X0vP79pOzp5p6kN5_A4nIorA8Oub4zXN229kcOaMYYxekO2jQ1Fph0Y-zsima4w',
    'CJw_pDg4GZ6ZfQES7i0hHSotEs3mEY4RtppyFVDuMs1lS5dd0FcUzNIT0ahMazOmSVEvXXuh8gEEFQ',
    'bkYuf64seREOvUvQHQW0L0Tra_Zy6wz6dHYHcq72NWIxgM8hztlpjoBbWGi7A3_WjackVqamqdsduw',
    'i1dT8GjNU0serv2XnwiPFFeiN0C-fKO_pUSPvWuGytfG4045h9EULmvuxzWL02qLqCvYiBcXBL06PA',
    '98rPYyP9AkiYmaYaE_RFC-XCp4yvoLLD2URnceCEcSiq_9WSuvUBRoRBARz0APGQH7sTeNHc7giTJQ',
    'n_kDs6Qe_PI9pj8Y4vQqZ3FNGt_bTsNERJTxlCe9AL9xKHIwKCz9zFyda19YzMVg5-WwHK9Pwvyvbg',
    'kCCX8W2QbLPbWIOqnpJzsifygrR4IxHPBeIG3VCKtSBeP-wa35nEahoP4ttKMjgenJGQpf1HNdyp9g',
    'BdFBALllvH6VXXBTirgSkvwmZsCeCgnPK_fOsYN0QdXh3o7r8JKOKbhOa8VyBhJtX-4l_L3AnzZNYg']},
  'info': {'game_datetime': 1658903684307,
   'game_length': 2120.8017578125,
   'game_version': 'Version 12.14.455.6722 (Jul 22 2022/14:23:11) [PUBLIC] <Releases/12.14>',
   'participants': [{'augmen

In [80]:
for result in matches_asset:
    print(result)
    break

In [42]:
results = matches_detail_collection.find({'_id': 'KR_6041556013'})

In [44]:
results

In [43]:
for result in results:
    print(result)

TypeError: 'NoneType' object is not iterable

In [35]:
results

In [37]:
all = summoners_collection.find({})

In [38]:
all

In [39]:
for result in all:
    print(result)

{'_id': '8ygNfnWPnYvCFKDJnKik0RNHxbOSveC8zrpq9jmYAIcLJw', 'accountId': 'oFecdwjzXmg2mfLvluM9G01HF9WcLuP1yTOSp0Z-pjVl', 'puuid': '5On3dmeoAoFhTS1wnsB4BoSm5FbUbbzSOL_JWjgoHo8LvjcFmjiHC9YPQX9y3eUlvVfSYPUW_IQkdg', 'name': 'ShrimpVicious', 'profileIconId': 4804, 'revisionDate': 1658850187000, 'summonerLevel': 312, 'summonerId': '8ygNfnWPnYvCFKDJnKik0RNHxbOSveC8zrpq9jmYAIcLJw', 'summonerName': 'ShrimpVicious', 'leaguePoints': 773, 'rank': 'I', 'wins': 203, 'losses': 169, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
{'_id': 'xDrh71lkeyZz1M433ne4OAoFgeLuRJhPGrByX6uZLL3jgQ', 'accountId': 'jLpfV6VipK19F69tka6DoWtqia56_dRuvaMnOlw4fQi4', 'puuid': 'QPbpIbefMxNWNOQDlWt5f2OkznBFmsd-lzu8wZgYFb02oMnVlcAvgA7XAZfoSMCHVYtrjpvx66bI_w', 'name': '띰 5', 'profileIconId': 4558, 'revisionDate': 1658892494000, 'summonerLevel': 414, 'summonerId': 'xDrh71lkeyZz1M433ne4OAoFgeLuRJhPGrByX6uZLL3jgQ', 'summonerName': '띰 5', 'leaguePoints': 748, 'rank': 'I', 'wins': 225, 'losses': 181, 'v